In [47]:
import influxdb_client
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
token = "lYtO30KjJmXRezB7nJaWbhUYHV7z9wKAtezFOmNyUtyakGE_Tlln5gMsWSwFy0S-lQSiqGtvhmRE3X7bbEABZw=="
org = "sgh"
url = "http://0.0.0.0:8086"

In [49]:
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [50]:
query_api = client.query_api()

In [53]:
query = """from(bucket: "aeroponics")
  |> range(start: 2024-11-01T10:21:00Z)
  |> filter(fn: (r) => r["_measurement"] == "first")
  |> filter(fn: (r) => r["_field"] == "humidity")
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")"""
tables = query_api.query(query, org="sgh")

In [54]:
data = []
for table in tables:
    for record in table.records:
        data.append({
            "datetime": record.get_time(),
            "airTemperature": record.get_value(),
        })
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime').asfreq("1min")
df

,airTemperature
datetime,
2024-12-16 05:13:00+00:00,58.327500
2024-12-16 05:14:00+00:00,52.215333
2024-12-16 05:15:00+00:00,49.499667
2024-12-16 05:16:00+00:00,53.409500
2024-12-16 05:17:00+00:00,46.871167
2024-12-16 05:18:00+00:00,49.645667
2024-12-16 05:19:00+00:00,44.098167
2024-12-16 05:20:00+00:00,44.675833
2024-12-16 05:21:00+00:00,47.833167
